# Edinburgh bike counter

Arjan Geers

Analysis of [Edinburgh bike counter data](https://data.edinburghopendata.info/dataset/bike-counter-data-set-cluster).

## Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import calendar
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

from bikecounter.data import get_edinburgh_bike_counter_data

In [ ]:
# Plot settings
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.style.use('seaborn')
TITLE_FONTSIZE = 16

%matplotlib inline

## Get data

Download and read the data of all bike counters. Each counter has one or more 'channels', which correspond to the cycling direction. However, the actual direction is not specified in the data. For now, we'll just consider the total count across all channels and forget about direction.

In [ ]:
hourly = get_edinburgh_bike_counter_data()
hourly.tail()

In [ ]:
# Define variables for later convenience
daily = hourly.resample('D').sum().replace(0, np.NaN)
counters = list(hourly.columns)
n_counters = len(counters)
n_cols = 5  # for grid plots
n_rows = math.ceil(n_counters / n_cols)  # for grid plots

## Bike counter operating days

When were the bike counters turned on and counting bikes?

In [ ]:
def get_datetime_ranges(datetimes, bin_size='D'):
    """Given a list of sorted datetimes, return a list
    of datetime ranges [(start, end), (...)] that
    correspond to periods without gaps at the bin_size
    resolution.
    
    """
    datetime_ranges = []
    start = datetimes[0]
    for i in range(1, len(datetimes)):
        if (datetimes[i].to_period(bin_size) -
            datetimes[i - 1].to_period(bin_size)).n > 1:
            end = datetimes[i - 1]
            datetime_ranges.append((start, end))
            start = datetimes[i]
    end = datetimes[-1]
    datetime_ranges.append((start, end))
    return datetime_ranges

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
for y, counter in enumerate(counters):
    operating_days = daily[counter].dropna().index
    operating_day_ranges = get_datetime_ranges(operating_days, bin_size='D')
    for operating_day_range in operating_day_ranges:
        ax.hlines(y, operating_day_range[0], operating_day_range[1])

ax.set_title('Bike counter operating days', fontsize=TITLE_FONTSIZE)
plt.yticks(range(n_counters), counters);

We see two distinct sets of bike counters. In early 2015, the first set stopped operating and the second started. Only two bike counters span the entire time period. We also see there are many large and small data gaps.

Are there any days of the week on which the bike counters were operating less (or more)?

In [ ]:
fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, 16))
for i, counter in enumerate(counters):
    current_ax = ax[i // n_cols][i % n_cols]
    operating_days = daily[counter].dropna().index
    operating_dayofweek_counts = operating_days.dayofweek.value_counts().sort_index()
    operating_dayofweek_counts.plot(ax=current_ax,
                                    kind='bar',
                                    title=counter,
                                    ylim=(0, 350))
    current_ax.set_xticklabels(list(calendar.day_abbr), rotation=0);
    
fig.suptitle('Distribution of operating days over the days of the week', y=1.02, fontsize=TITLE_FONTSIZE)
plt.tight_layout()

All days appear to be equally represented in the dataset, so we can just average over all available data to look at hourly and daily trends.

## Hourly and daily trends

Let's look at the hourly trend first.

In [ ]:
hourly_trend = hourly.pivot_table(index=hourly.index.hour, aggfunc='mean')

fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, 16))
for i, counter in enumerate(counters):
    current_ax = ax[i // n_cols][i % n_cols]
    hourly_trend[counter].plot(ax=current_ax,
                               title=counter,
                               ylim=(0, 125))
    
fig.suptitle('Hourly trend of bike activity', y=1.02, fontsize=TITLE_FONTSIZE)
plt.tight_layout()

By keeping the limits of the y-axis the same, we can easily see that some roads are much busier than others. Nearly all roads show a bimodal traffic pattern with a peak in the morning and a peak in the afternoon, corresponding to rush hour. On Bruntsfield Place, the separate counters for the northbound (into the city) and southbound (out of the city) directions show relatively large peaks going into the city in the morning and out of the city in the afternoon, which matches our expectation of commuting flows. If we were to look further into cycling direction, we would have to consider the bike counter channels separately and 'guess' the direction.

How about the daily trend?

In [ ]:
daily_trend = daily.pivot_table(index=daily.index.dayofweek, aggfunc='mean')

fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, 16))
for i, counter in enumerate(counters):
    current_ax = ax[i // n_cols][i % n_cols]
    daily_trend[counter].plot(ax=current_ax,
                              kind='bar',
                              title=counter)
    current_ax.set_xticklabels(list(calendar.day_abbr), rotation=0);
    
fig.suptitle('Daily trend of bike activity', y=1.02, fontsize=TITLE_FONTSIZE)
plt.tight_layout()

Note that we use flexible y-axis limits to highlight the relative variation throughout the week. We see that for most counters the bike activity is much higher on weekdays than in the weekend. Silverknowes, Cramond, and Dalmeny - in a lovely area to the northwest of Edinburgh - are more popular with cyclists in the weekend. 

## Unsupervised clustering

In this section, we will closely follow the approach described [here](https://jakevdp.github.io/blog/2015/07/23/learning-seattles-work-habits-from-bicycle-counts/) to cluster days of data measured by one of the bike counters. The example we'll pick is from Portobello Promenade, a cycle(able) path along the beach that I occasionally used to get to and from work.

Note that we'll reuse some of the variable names in the code. `hourly` will from now on contain data of only one counter instead of all of them.

In [ ]:
COUNTER_NAME = '20portobellopromeast'

hourly_all = hourly.copy()  # keep original dataset with all counters
hourly = pd.DataFrame(hourly_all[COUNTER_NAME].dropna())

First we transform the data to a 2D matrix with days as rows and hours as columns.

In [ ]:
hourly_pivoted = hourly.pivot_table(index=hourly.index.date,
                                    columns=hourly.index.hour,
                                    fill_value=0)
X = hourly_pivoted.values

We then reduce the dimensionality of the matrix from 24 to 2 using principal component analysis (PCA), so we can create a scatter plot and visually look for clusters. As you can see below, the 2 components we get still preserve 81% of the variance in the dataset.

In [ ]:
pca = PCA(2)
Xpca = pca.fit_transform(X)
print(f'Preserved variance with 2 components: {pca.explained_variance_ratio_.sum():.3f}')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(Xpca[:, 0], Xpca[:, 1]);

We see that the days can be nicely split in two clusters. We'll use a Gaussian Mixture model to automatically label the points.

In [ ]:
gmm = GaussianMixture(2, covariance_type='full', random_state=0)
gmm.fit(Xpca)
cluster_label = gmm.predict(Xpca)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(Xpca[:, 0], Xpca[:, 1], c=cluster_label, cmap='Set1');

Not bad. The red points belong to cluster 0, the gray ones to cluster 1. Let's see what the hourly trends look like for the two clusters.  

In [ ]:
hourly_pivoted['cluster'] = cluster_label
hourly_labeled = hourly.join(hourly_pivoted['cluster'], on=hourly.index.date)
hourly_trend = hourly_labeled.groupby(['cluster', hourly_labeled.index.hour]).mean()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

for i in range(2):
    hourly_trend.loc[i].plot(ax=ax[i], legend=False)
    ax[i].set_title(f'cluster {i}')

Nice! Cluster 0 shows the typical commuting flows from weekdays while cluster 1 shows the more recreational cycling you'd expect to see in the weekend. We can confirm this by coloring the scatter plot points by day of the week. 

In [ ]:
dayofweek = pd.to_datetime(hourly_pivoted.index).dayofweek

fig, ax = plt.subplots(figsize=(8, 6))
s = ax.scatter(Xpca[:, 0], Xpca[:, 1], c=dayofweek,
               cmap=plt.cm.get_cmap('jet', 7))
cb = fig.colorbar(s, ticks=range(7))
cb.set_ticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
s.set_clim(-0.5, 6.5);

Cluster 1 is indeed dominated by weekend days.